# Interaction Model Examples

<a href="http://35.236.121.59/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fproject-chip%2Fconnectedhomeip&urlpath=lab%2Ftree%2Fconnectedhomeip%2Fdocs%2Fguides%2Frepl%2FMatter%2520-%2520Basic%2520Interactions.ipynb&branch=master">
<img src="https://i.ibb.co/hR3yWsC/launch-playground.png" alt="drawing" width="130"/>
</a>
<br></br>

This walks through the various interactions that can be initiated from the REPL towards a target using the Matter Interaction Model (IM) and Data Model (DM).

## Clear Persisted Storage

Let's clear out our persisted storage (if one exists) to start from a clean slate.

In [1]:
import os, subprocess

if os.path.isfile('/tmp/repl-storage.json'):
    os.remove('/tmp/repl-storage.json')

# So that the all-clusters-app won't boot with stale prior state.
os.system('rm -rf /tmp/chip_*')

0

## Initialization

Let's first begin by setting up by importing some key modules that are needed to make it easier for us to interact with the Matter stack.

`ChipReplStartup.py` is run within the global namespace. This results in all of its imports being made available here.

> **NOTE**: _This is not needed if you launch the REPL from the command-line._

In [ ]:
import chip.native
import pkgutil
module = pkgutil.get_loader('chip.ChipReplStartup')
%run {module.path}

## Cluster Elements

The Interaction Model uses data model types that refer not just to the various base types defines in the spec, but types that correspond to structs/commands/events/attributes defined in each cluster specification. The cluster-specific types are referred to as 'cluster objects'. These are represented as Python dataclasses, with each field in the respective object equivalently named as a member within the dataclass.

### Namespaces

Objects in clusters are organized into namespaces. All clusters can be found under the `Clusters` namespace, with the appropriate cluster in upper camel case within that. (e.g `Clusters.UnitTesting`).

Within that, `Commands`, `Structs` and `Attributes` delimit the respective types in the cluster.

_Example Struct:_

In [ ]:
v = Clusters.UnitTesting.Structs.SimpleStruct()
v.a = 20
v.b = True
v.c = Clusters.UnitTesting.Enums.SimpleEnum.kValueA
v.d = b'1234'
v.e = 30
v.g = 23.234

v

SimpleStruct(
│   a=20,
│   b=True,
│   c=<SimpleEnum.kValueA: 1>,
│   d=b'1234',
│   e=30,
│   f=0,
│   g=23.234,
│   h=0.0
)

_Example Command:_

In [ ]:
Clusters.UnitTesting.Commands.TestAddArguments()

TestAddArguments(
│   arg1=0,
│   arg2=0
)

To get more information about the fields in these objects and their types, run:

In [ ]:
matterhelp(Clusters.UnitTesting.Commands.TestAddArguments)

╭──────── <class 'chip.clusters.Objects.UnitTesting.Commands.TestAddArguments'> ─────────╮
│ def TestCluster.Commands.TestAddArguments(arg1: 'uint' = 0, arg2: 'uint' = 0) -> None: │
│                                                                                        │
│ TestAddArguments(arg1: 'uint' = 0, arg2: 'uint' = 0)                                   │
│                                                                                        │
│                  arg1 = 0                                                              │
│                  arg2 = 0                                                              │
│            cluster_id = 1295                                                           │
│            command_id = 4                                                              │
│            descriptor = ClusterObjectDescriptor(                                       │
│                             Fields=[                                                   │
│                                 ClusterObjectFieldDescriptor(                          │
│                                     Label='arg1',                                      │
│                                     Tag=0,                                             │
│                                     Type=<class 'chip.tlv.uint'>                       │
│                                 ),                                                     │
│                                 ClusterObjectFieldDescriptor(                          │
│                                     Label='arg2',                                      │
│                                     Tag=1,                                             │
│                                     Type=<class 'chip.tlv.uint'>                       │
│                                 )                                                      │
│                             ]                                                          │
│                         )                                                              │
│             is_client = True                                                           │
│ must_use_timed_invoke = False                                                          │
│              FromDict = def FromDict(data: dict):                                      │
│               FromTLV = def FromTLV(data: bytes):                                      │
│                 ToTLV = def ToTLV(self):                                               │
╰────────────────────────────────────────────────────────────────────────────────────────╯

### Nullable Fields

For fields that are nullable, they are represented as a `Typing.Union[Nullable, ...]`. This means that it can either be a `Nullable` type or the underlying type of the field.

When nullable, a field can either take on the value of the native type, or a value of `NullValue`.

In [ ]:
a = Clusters.UnitTesting.Structs.NullablesAndOptionalsStruct()
a.nullableInt = Clusters.Types.NullValue
a

NullablesAndOptionalsStruct(
│   nullableInt=Null,
│   optionalInt=None,
│   nullableOptionalInt=None,
│   nullableString=Null,
│   optionalString=None,
│   nullableOptionalString=None,
│   nullableStruct=Null,
│   optionalStruct=None,
│   nullableOptionalStruct=None,
│   nullableList=Null,
│   optionalList=None,
│   nullableOptionalList=None
)

### Optional Fields

If a field is optional, it is represented in the typing hints as a `Typing.Union[NoneType, ...]`. An optional field that isn't present has a value of `None`.

In [ ]:
print(a.nullableOptionalInt)

None

### Defaults

Upon construction of a cluster object, the fields within that object are automatically initialized to type specific defaults as specified in the data model specification:

In [ ]:
Clusters.UnitTesting.Structs.SimpleStruct()

SimpleStruct(
│   a=0,
│   b=False,
│   c=0,
│   d=b'',
│   e='',
│   f=0,
│   g=0.0,
│   h=0.0
)

## IM Interactions

This section will walk through the various types of IM Interactions that are possible in the REPL.

### Commission and Setup Server

#### Launch Server

Let's launch an instance of the `chip-all-clusters-app`.

> NOTE: If you're interacting with real devices, this step can be skipped.

In [ ]:
import time, os
import subprocess

# So that the all-clusters-app won't boot with stale prior state.    
os.system('pkill -f chip-all-clusters-app')

# The location of the all-clusters-app in the cloud playground is one level higher - adjust for this by testing for file presence.
if (os.path.isfile('../../../out/debug/chip-all-clusters-app')):
    appPath = '../../../out/debug/chip-all-clusters-app'
else:
    appPath = '../../../../out/debug/chip-all-clusters-app'
    
process = subprocess.Popen(appPath, stdout=subprocess.DEVNULL)
time.sleep(1)

#### Discover and commission commissionable node

In [6]:
devices = devCtrl.DiscoverCommissionableNodes(filterType=chip.discovery.FilterType.LONG_DISCRIMINATOR, filter=3840, stopOnFirst=True, timeoutSecond=2)
devices

[
│   CommissionableNode(
│   │   (To Be Commissioned By)='caIndex(1)/fabricId(0x0000000000000001)/nodeId(0x000000000001B669)',
│   │   CommissionableNode(
│   │   │   instanceName='B2D5B11D799BB1E0',
│   │   │   hostName='DCA63289ABCD0000',
│   │   │   port=5540,
│   │   │   longDiscriminator=3840,
│   │   │   vendorId=65521,
│   │   │   productId=32769,
│   │   │   commissioningMode=1,
│   │   │   deviceType=0,
│   │   │   deviceName='',
│   │   │   pairingInstruction='',
│   │   │   pairingHint=33,
│   │   │   mrpRetryIntervalIdle=5000,
│   │   │   mrpRetryIntervalActive=300,
│   │   │   supportsTcp=True,
│   │   │   addresses=[
│   │   │   │   'fd1e:1a94:d591:6914:dea6:32ff:fe89:abcd',
│   │   │   │   'fe80::dea6:32ff:fe89:abcd',
│   │   │   │   '192.168.2.1'
│   │   │   ]
│   │   )
│   )
]

You can find a list of discovered device

You can call `Commission(nodeId, setupPinCode)` on one of the returned object:

In [ ]:
devices[0].Commission(233, 20202021)

The device will be commissioned by the DeviceController instance that discovered it (`caIndex(1)/fabricId(0x0000000000000001)/nodeId(0x000000000001B669)` in this case).

#### Commission Target (Locally Launched App)

Commission the target with a NodeId of 1.

In [4]:
devCtrl.CommissionIP(b'127.0.0.1', 20202021, 2)

2022-01-29 15:34:45 johnsj-macbookpro1.roam.corp.google.com chip.SC[10607] ERROR The device does not support GetClock_RealTimeMS() API. This will eventually result in CASE session setup failures.


Node address has been updated
Commissioning complete


True

#### Commission Target (BLE + Thread)

To commission a Thread-based target over BLE, ensure your BLE stack is up on your host and available as `hci0` on Linux. You can confirm this by running `hciconfig -a`. You'll also need Thread credentials to join the Thread network.

> NOTE: MacOS Monterey is currently not supported due to issues with its BLE stack.

In [ ]:
devCtrl.CommissionThread(3840, 20202021, 2, b'\x01\x03\xff')

#### Commission Target (BLE + WiFi)

To commission a Wifi-based target over BLE, ensure your BLE stack is up on your host and available as `hci0` on Linux. You can confirm this by running `hciconfig -a`. You'll also need Wifi credentials to join the Thread network.

> NOTE: MacOS Monterey is currently not supported due to issues with its BLE stack.

In [ ]:
devCtrl.CommissionWiFi(3840, 20202021, 2, 'MyWifiSsid', 'MyWifiPassword')

### Invoke Interaction

#### Basic Command (Success Response)

Let's send a basic command to turn on/off the light on Endpoint 1.

In [5]:
await devCtrl.SendCommand(2, 1, Clusters.OnOff.Commands.On())

The receipt of a successful status response will result in the command just returning successfully. Otherwise, an exception will be thrown.

#### Basic Command (Failure Response)

If we send the same command to an invalid endpoint, an exception is thrown. If an IM status code was received from the server, a `InteractionModelError` is thrown containing the IM status code:

In [11]:
try:
    await devCtrl.SendCommand(2, 100, Clusters.OnOff.Commands.On())
except Exception as e:
    pprint(e)

InteractionModelError(<Status.Failure: 1>)

#### Basic Command (Data Response)

Here's an example of a command that sends back a data response, and how that is presented:

In [12]:
await devCtrl.SendCommand(2, 1, Clusters.UnitTesting.Commands.TestListInt8UReverseRequest([1, 3, 5, 7]))

TestListInt8UReverseResponse(
│   arg1=[
│   │   7,
│   │   5,
│   │   3,
│   │   1
│   ]
)

### Read Interaction

The `ReadAttribute` method on the `DeviceController` class can be used to read attributes from a target. The NodeId of the target is the first argument, followed by a list of paths that are expressed as cluster object namespaces to the respective slices of the data that is requested.

By default, the data is returned as a dictionary, with the top-level item representing the endpoint, then the cluster and the attribute. The latter two keys are expressed using cluster object namespaces.

#### Read 1 attribute:

In [13]:
a = await devCtrl.ReadAttribute(2, [Clusters.UnitTesting.Attributes.Int16u])
a

{
│   1: {
│   │   <class 'chip.clusters.Objects.UnitTesting'>: {
│   │   │   <class 'chip.clusters.Objects.UnitTesting.Attributes.Int16u'>: 0
│   │   }
│   }
}

In [14]:
a[1][Clusters.UnitTesting]

{
│   <class 'chip.clusters.Objects.UnitTesting.Attributes.Int16u'>: 0
}

In [15]:
a[1][Clusters.UnitTesting][Clusters.UnitTesting.Attributes.Int16u]

0

#### Read 2 attributes:

In [16]:
await devCtrl.ReadAttribute(2, [Clusters.UnitTesting.Attributes.Int16u, Clusters.UnitTesting.Attributes.Boolean])

{
│   1: {
│   │   <class 'chip.clusters.Objects.UnitTesting'>: {
│   │   │   <class 'chip.clusters.Objects.UnitTesting.Attributes.Boolean'>: False,
│   │   │   <class 'chip.clusters.Objects.UnitTesting.Attributes.Int16u'>: 0
│   │   }
│   }
}

#### Read the entirety of a cluster on an endpoint:

The path is represented as tuple of (endpoint, cluster)

In [17]:
await devCtrl.ReadAttribute(2, [(1, Clusters.OnOff)])

{
│   1: {
│   │   <class 'chip.clusters.Objects.OnOff'>: {
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.StartUpOnOff'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.FeatureMap'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.ClusterRevision'>: 4
│   │   }
│   }
}

#### Read the entirety of a cluster across all endpoints:

In [18]:
await devCtrl.ReadAttribute(2, [Clusters.OnOff])

{
│   1: {
│   │   <class 'chip.clusters.Objects.OnOff'>: {
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.StartUpOnOff'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.FeatureMap'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.ClusterRevision'>: 4
│   │   }
│   },
│   2: {
│   │   <class 'chip.clusters.Objects.OnOff'>: {
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>: False,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.StartUpOnOff'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.FeatureMap'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.ClusterRevision'>: 4
│   │   }
│   }
}

#### Read an endpoint:

In [19]:
await devCtrl.ReadAttribute(2, [2])

{
│   2: {
│   │   <class 'chip.clusters.Objects.Groups'>: {
│   │   │   <class 'chip.clusters.Objects.Groups.Attributes.NameSupport'>: 128,
│   │   │   <class 'chip.clusters.Objects.Groups.Attributes.ClusterRevision'>: 3
│   │   },
│   │   <class 'chip.clusters.Objects.OnOff'>: {
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>: False,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>: True,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.StartUpOnOff'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.FeatureMap'>: 0,
│   │   │   <class 'chip.clusters.Objects.OnOff.Attributes.ClusterRevision'>: 4
│   │   },
│   │   <class 'chip.clusters.Objects.Descriptor'>: {
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.DeviceList'>: [
│   │   │   │   DeviceType(
│   │   │   │   │   type=256,
│   │   │   │   │   revision=1
│   │   │   │   )
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.ServerList'>: [
│   │   │   │   4,
│   │   │   │   6,
│   │   │   │   29,
│   │   │   │   1030
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.ClientList'>: [],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.PartsList'>: [],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.ClusterRevision'>: 1
│   │   },
│   │   <class 'chip.clusters.Objects.OccupancySensing'>: {
│   │   │   <class 'chip.clusters.Objects.OccupancySensing.Attributes.Occupancy'>: 0,
│   │   │   <class 'chip.clusters.Objects.OccupancySensing.Attributes.OccupancySensorType'>: 0,
│   │   │   <class 'chip.clusters.Objects.OccupancySensing.Attributes.OccupancySensorTypeBitmap'>: 1,
│   │   │   <class 'chip.clusters.Objects.OccupancySensing.Attributes.ClusterRevision'>: 2
│   │   }
│   }
}

#### Read the entire node:

In [20]:
await devCtrl.ReadAttribute(2, [('*')])

2022-01-25 16:58:32 johnsj-macbookpro1.roam.corp.google.com root[27801] ERROR For path: Endpoint = 0, Attribute = <class 'chip.clusters.Objects.NetworkCommissioning.Attributes.Networks'>, got IM Error: InteractionModelError: UnsupportedRead (0x8f)
2022-01-25 16:58:32 johnsj-macbookpro1.roam.corp.google.com root[27801] ERROR For path: Endpoint = 1, Attribute = <class 'chip.clusters.Objects.NetworkCommissioning.Attributes.Networks'>, got IM Error: InteractionModelError: UnsupportedRead (0x8f)
2022-01-25 16:58:32 johnsj-macbookpro1.roam.corp.google.com root[27801] ERROR For path: Endpoint = 1, Attribute = <class 'chip.clusters.Objects.UnitTesting.Attributes.GeneralErrorBoolean'>, got IM Error: InteractionModelError: InvalidDataType (0x8d)
2022-01-25 16:58:32 johnsj-macbookpro1.roam.corp.google.com root[27801] ERROR For path: Endpoint = 1, Attribute = <class 'chip.clusters.Objects.UnitTesting.Attributes.ClusterErrorBoolean'>, got IM Error: InteractionModelError: Failure (0x1)


{
│   0: {
│   │   <class 'chip.clusters.Objects.Identify'>: {
│   │   │   <class 'chip.clusters.Objects.Identify.Attributes.IdentifyTime'>: 0,
│   │   │   <class 'chip.clusters.Objects.Identify.Attributes.IdentifyType'>: 2,
│   │   │   <class 'chip.clusters.Objects.Identify.Attributes.ClusterRevision'>: 2
│   │   },
│   │   <class 'chip.clusters.Objects.Groups'>: {
│   │   │   <class 'chip.clusters.Objects.Groups.Attributes.NameSupport'>: 128,
│   │   │   <class 'chip.clusters.Objects.Groups.Attributes.ClusterRevision'>: 3
│   │   },
│   │   <class 'chip.clusters.Objects.Descriptor'>: {
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.DeviceList'>: [
│   │   │   │   DeviceType(
│   │   │   │   │   type=22,
│   │   │   │   │   revision=1
│   │   │   │   )
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.ServerList'>: [
│   │   │   │   3,
│   │   │   │   4,
│   │   │   │   29,
│   │   │   │   30,
│   │   │   │   31,
│   │   │   │   40,
│   │   │   │   42,
│   │   │   │   43,
│   │   │   │   44,
│   │   │   │   46,
│   │   │   │   48,
│   │   │   │   49,
│   │   │   │   50,
│   │   │   │   51,
│   │   │   │   52,
│   │   │   │   53,
│   │   │   │   54,
│   │   │   │   55,
│   │   │   │   60,
│   │   │   │   62,
│   │   │   │   63,
│   │   │   │   64,
│   │   │   │   65,
│   │   │   │   1029
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.ClientList'>: [
│   │   │   │   41
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.PartsList'>: [
│   │   │   │   1,
│   │   │   │   2
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.Descriptor.Attributes.ClusterRevision'>: 1
│   │   },
│   │   <class 'chip.clusters.Objects.Binding'>: {
│   │   │   <class 'chip.clusters.Objects.Binding.Attributes.ClusterRevision'>: 1
│   │   },
│   │   <class 'chip.clusters.Objects.AccessControl'>: {
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.Acl'>: [
│   │   │   │   AccessControlEntry(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   privilege=<Privilege.kAdminister: 5>,
│   │   │   │   │   authMode=<AuthMode.kCase: 2>,
│   │   │   │   │   subjects=[
│   │   │   │   │   │   1
│   │   │   │   │   ],
│   │   │   │   │   targets=Null
│   │   │   │   )
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.Extension'>: [],
│   │   │   <class 'chip.clusters.Objects.AccessControl.Attributes.ClusterRevision'>: 1
│   │   },
│   │   <class 'chip.clusters.Objects.Basic'>: {
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.InteractionModelVersion'>: 1,
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.VendorName'>: 'TEST_VENDOR',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.VendorID'>: 9050,
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.ProductName'>: 'TEST_PRODUCT',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.ProductID'>: 65279,
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.NodeLabel'>: '',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.Location'>: 'XX',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.HardwareVersion'>: 0,
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.HardwareVersionString'>: 'TEST_VERSION',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.SoftwareVersion'>: 0,
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.SoftwareVersionString'>: 'prerelease',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.ManufacturingDate'>: '20210614123456ZZ',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.PartNumber'>: '',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.ProductURL'>: '',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.ProductLabel'>: '',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.SerialNumber'>: 'TEST_SN',
│   │   │   <class 'chip.clusters.Objects.Basic.Attributes.LocalConfigDisabled'>: False,
│   │   │  

#### Alternative 'Cluster' View

The above encapsulates each attribute as a 'cluster-object' key within the top-level cluster instance. Instead, an alternative view each attribute is represented as a field in the object can be retrieved by passing in `True` to the third argument:

In [21]:
await devCtrl.ReadAttribute(2, [2], True)

{
│   2: {
│   │   <class 'chip.clusters.Objects.Groups'>: Groups(
│   │   │   nameSupport=128,
│   │   │   attributeList=None,
│   │   │   featureMap=None,
│   │   │   clusterRevision=3
│   │   ),
│   │   <class 'chip.clusters.Objects.OnOff'>: OnOff(
│   │   │   onOff=False,
│   │   │   globalSceneControl=True,
│   │   │   onTime=0,
│   │   │   offWaitTime=0,
│   │   │   startUpOnOff=0,
│   │   │   attributeList=None,
│   │   │   featureMap=0,
│   │   │   clusterRevision=4
│   │   ),
│   │   <class 'chip.clusters.Objects.Descriptor'>: Descriptor(
│   │   │   deviceList=[
│   │   │   │   DeviceType(
│   │   │   │   │   type=256,
│   │   │   │   │   revision=1
│   │   │   │   )
│   │   │   ],
│   │   │   serverList=[
│   │   │   │   4,
│   │   │   │   6,
│   │   │   │   29,
│   │   │   │   1030
│   │   │   ],
│   │   │   clientList=[],
│   │   │   partsList=[],
│   │   │   attributeList=None,
│   │   │   featureMap=None,
│   │   │   clusterRevision=1
│   │   ),
│   │   <class 'chip.clusters.Objects.OccupancySensing'>: OccupancySensing(
│   │   │   occupancy=0,
│   │   │   occupancySensorType=0,
│   │   │   occupancySensorTypeBitmap=1,
│   │   │   pirOccupiedToUnoccupiedDelay=None,
│   │   │   pirUnoccupiedToOccupiedDelay=None,
│   │   │   pirUnoccupiedToOccupiedThreshold=None,
│   │   │   ultrasonicOccupiedToUnoccupiedDelay=None,
│   │   │   ultrasonicUnoccupiedToOccupiedDelay=None,
│   │   │   ultrasonicUnoccupiedToOccupiedThreshold=None,
│   │   │   physicalContactOccupiedToUnoccupiedDelay=None,
│   │   │   physicalContactUnoccupiedToOccupiedDelay=None,
│   │   │   physicalContactUnoccupiedToOccupiedThreshold=None,
│   │   │   attributeList=None,
│   │   │   featureMap=None,
│   │   │   clusterRevision=2
│   │   )
│   }
}

#### Read Events:

A `ReadEvents` API exists that behaves similarly to the `ReadAttributes` API. It permits the same degrees of wildcard expression as its counterpart and follows the same format for expressing all wildcard permutations.

#### Read all events:

In [22]:
# Force an event to get emitted.
await devCtrl.SendCommand(2, 1, Clusters.UnitTesting.Commands.TestEmitTestEventRequest())

await devCtrl.ReadEvent(2, [('*')])

[
│   EventReadResult(
│   │   Header=EventHeader(
│   │   │   EndpointId=1,
│   │   │   Event=None,
│   │   │   EventNumber=0,
│   │   │   Priority=<EventPriority.DEBUG: 1>,
│   │   │   Timestamp=<EventTimestampType.SYSTEM: 0>,
│   │   │   TimestampType=None
│   │   ),
│   │   Status=<Status.Success: 0>,
│   │   Data=TestEvent(
│   │   │   arg1=0,
│   │   │   arg2=<SimpleEnum.kUnspecified: 0>,
│   │   │   arg3=False,
│   │   │   arg4=SimpleStruct(
│   │   │   │   a=0,
│   │   │   │   b=False,
│   │   │   │   c=<SimpleEnum.kUnspecified: 0>,
│   │   │   │   d=b'',
│   │   │   │   e='',
│   │   │   │   f=0,
│   │   │   │   g=0.0,
│   │   │   │   h=0.0
│   │   │   ),
│   │   │   arg5=[],
│   │   │   arg6=[]
│   │   )
│   )
]

### Subscription Interaction

To subscribe to a Node, the same `ReadAttributes` API is used to trigger a subscription, with a valid `reportInterval` tuple passed in being used as a way to indicate the request to create a subscription.

In [23]:
reportingTimingParams = (0, 2) # MinInterval = 0s, MaxInterval = 2s
subscription = await devCtrl.ReadAttribute(2, [(2, Clusters.OnOff)], True, reportingTimingParams)
subscription

<Subscription (Id=6469889299346410316)>

In [24]:
subscription.GetAttributes()

{
│   2: {
│   │   <class 'chip.clusters.Objects.OnOff'>: OnOff(
│   │   │   onOff=False,
│   │   │   globalSceneControl=True,
│   │   │   onTime=0,
│   │   │   offWaitTime=0,
│   │   │   startUpOnOff=0,
│   │   │   attributeList=None,
│   │   │   featureMap=0,
│   │   │   clusterRevision=4
│   │   )
│   }
}

#### Trigger Report

To trigger a report, let's alter the state of the on/off switch on EP1. That should trigger the generation of a set of attribute reports.

The `SubscriptionTransaction` object returned by `ReadAttribute` permits installing a callback that is invoked on any attribute report. A default callback is installed above that just dumps out the attribute data.

In [25]:
await devCtrl.SendCommand(2, 2, Clusters.OnOff.Commands.On())
time.sleep(1)

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>,
│   'Value': True
}

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.GlobalSceneControl'>,
│   'Value': True
}

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.OffWaitTime'>,
│   'Value': 0
}

In [26]:
await devCtrl.SendCommand(2, 2, Clusters.OnOff.Commands.Off())
time.sleep(1)

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.OnOff'>,
│   'Value': False
}

Attribute Changed:


{
│   'Endpoint': 2,
│   'Attribute': <class 'chip.clusters.Objects.OnOff.Attributes.OnTime'>,
│   'Value': 0
}

In [27]:
subscription.Shutdown()

#### Subscribe to Events

In [28]:
reportingTimingParams = (0, 2) # MinInterval = 0s, MaxInterval = 2s
subscription = await devCtrl.ReadEvent(2, [()], reportingTimingParams)

In [29]:
subscription.GetEvents()

[
│   EventReadResult(
│   │   Header=EventHeader(
│   │   │   EndpointId=1,
│   │   │   Event=None,
│   │   │   EventNumber=0,
│   │   │   Priority=<EventPriority.DEBUG: 1>,
│   │   │   Timestamp=<EventTimestampType.SYSTEM: 0>,
│   │   │   TimestampType=None
│   │   ),
│   │   Status=<Status.Success: 0>,
│   │   Data=TestEvent(
│   │   │   arg1=0,
│   │   │   arg2=<SimpleEnum.kUnspecified: 0>,
│   │   │   arg3=False,
│   │   │   arg4=SimpleStruct(
│   │   │   │   a=0,
│   │   │   │   b=False,
│   │   │   │   c=<SimpleEnum.kUnspecified: 0>,
│   │   │   │   d=b'',
│   │   │   │   e='',
│   │   │   │   f=0,
│   │   │   │   g=0.0,
│   │   │   │   h=0.0
│   │   │   ),
│   │   │   arg5=[],
│   │   │   arg6=[]
│   │   )
│   )
]

### Trigger Event

Force an event to get emitted, which after a short while, should generate a report and trigger the print out of the received event:

In [30]:
await devCtrl.SendCommand(2, 1, Clusters.UnitTesting.Commands.TestEmitTestEventRequest())
time.sleep(3)

Received Event:


EventReadResult(
│   Header=EventHeader(
│   │   EndpointId=1,
│   │   Event=None,
│   │   EventNumber=1,
│   │   Priority=<EventPriority.DEBUG: 1>,
│   │   Timestamp=<EventTimestampType.SYSTEM: 0>,
│   │   TimestampType=None
│   ),
│   Status=<Status.Success: 0>,
│   Data=TestEvent(
│   │   arg1=0,
│   │   arg2=<SimpleEnum.kUnspecified: 0>,
│   │   arg3=False,
│   │   arg4=SimpleStruct(
│   │   │   a=0,
│   │   │   b=False,
│   │   │   c=<SimpleEnum.kUnspecified: 0>,
│   │   │   d=b'',
│   │   │   e='',
│   │   │   f=0,
│   │   │   g=0.0,
│   │   │   h=0.0
│   │   ),
│   │   arg5=[],
│   │   arg6=[]
│   )
)

In [31]:
subscription.Shutdown()

### Write Interaction

To write attribute data, the `WriteAttribute` API can be used. It requires a NodeId and a list of cluster object encapsulated data for the attribute being written.

In [32]:
await devCtrl.WriteAttribute(2, [ (1, Clusters.UnitTesting.Attributes.Int16u(2)) ])

[
│   AttributeStatus(
│   │   Path=AttributePath(
│   │   │   EndpointId=1,
│   │   │   ClusterId=1295,
│   │   │   AttributeId=6
│   │   ),
│   │   Status=<Status.Success: 0>
│   )
]

In [33]:
await devCtrl.ReadAttribute(2, [ (1, Clusters.UnitTesting.Attributes.Int16u) ])

{
│   1: {
│   │   <class 'chip.clusters.Objects.UnitTesting'>: {
│   │   │   <class 'chip.clusters.Objects.UnitTesting.Attributes.Int16u'>: 2
│   │   }
│   }
}